## importing libraries

In [ ]:
import pandas as pd
import numpy as np
import random as rd
from sklearn import preprocessing
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

## generating data

In [ ]:
genes = ['gene' + str(i) for i in range(1, 10001)]
wt = ['wt' + str(i) for i in range(1, 6)]
ko = ['ko' + str(i) for i in range(1, 6)]

## creating the dataframe to store the data

In [ ]:
data = pd.DataFrame(columns = [*wt, *ko], index = genes)

### the star unpack the wt, ko, so that the columns names are in a single array that looks like this: 
[wt1, wt2, wt3, wt4, ....., ko1, ko2, ko3...., ko6]
### Without the stars we would have created an array of two arrays, and they wouldn't create 12 columns like we want:
[[wt1, wt2, wt3, .....],[ko1, ko2, ko3....]]

In [ ]:
for gene in data.index:
    data.loc[gene, 'wt1':'wt5'] = np.random.poisson(lam = rd.randrange(10, 1000), size = 5)
    data.loc[gene, 'ko1':'ko5'] = np.random.poisson(lam = rd.randrange(10, 1000), size = 5)

### np.random.poisson, implies that made up data come from two poisson distributions: one for wt and one for ko samples

## Lookup the data

In [ ]:
data.head()

In [ ]:
data.shape

## Center and Scale the data

### Using sklearn, we can scale and center the data so that the mean of each gene is 0 and standard deviation of each gene is 1.
### Alternatively, we could have used:
      StandardScalar().fit_transform(data.T)

For scaling the data, the method prefers to have the data in rows instead of columns. So, ensure that the data is stored row wise, otherwise use tranpose of the data.

### After centering the mean of each gene will be 0 and after scalling the standard deviation for the values of each gene will be 1

In [ ]:
## data.T is basically is transformation of the data
scaled_data = preprocessing.scale(data.T)

In sklearn, the standard variation is calculated like, 
    
```
[(measurements - mean)^2]/(the number of measurements)
```
In R, using scale(), or prcomp(), variation is calculated as:

```
[(measurements - mean)^2]/(the number of measurements - 1)
```
Though, they don't make any difference in PCA analysis[good news]

The bad news, these differences will have a minor effect the final graph.
Because, the coordinates on the final graph come from multiplying the loading scores by the scaled values. 

## creating object for sklearn 

In [ ]:
pca = PCA()

In [ ]:
pca.fit(scaled_data)
pca_data = pca.transform(scaled_data)  # this is where we generate coordinates for the graph

## we calculate the percentage of variation that each principle component accounts for

In [ ]:
per_var = np.round(pca.explained_variance_ratio_*100, decimals = 1)

## creating lables for scree plot

In [ ]:
labels = ['PC' + str(i) for i in range(1, len(per_var)+1)]

## plotting the graph

In [ ]:
plt.bar(x = range(1, len(per_var)+1), height = per_var, tick_label = labels)
plt.ylabel("Percentage of Explained Variance")
plt.xlabel("Principal Component")
plt.title("Scree Plot")
plt.show()

## drawing a pca plot

we will put the new cordinates, created by pca.transform(scaled.data), into a nice matrix where the rows have sample labels, and the columns have PC labels.

In [ ]:
pca_df = pd.DataFrame(pca_data, index = [*wt, *ko], columns = labels)

## scatter plot

In [ ]:
plt.scatter(pca_df.PC1, pca_df.PC2)
plt.title("PCA Graph")
plt.xlabel('PC1 - {0}%'.format(per_var[0]))
plt.ylabel('PC2 - {0}%'.format(per_var[1]))

## loop genertaes the name of the samples
for sample in pca_df.index:
    plt.annotate(sample, (pca_df.PC1.loc[sample], pca_df.PC2.loc[sample]))

plt.show()    

Observation from PCA plot:
    
    1. wt samples are correlated with each other and ko samples are co-related with each other.
    2. The seperation between wt and ko,samples prove that they are different from each other.  

### Looking at the loading scores to gain some insights, which genes had influenced most on seperating the two clusters along the x-axis.

In [ ]:
# we will start by creating pandas series.
loading_scores = pd.Series(pca.components_[0], index = genes) 
sorted_loading_scores = loading_scores.abs().sort_values(ascending = False)

# top 10 indexes
top_10_genes = sorted_loading_scores[0:10].index.values

print(loading_scores[top_10_genes])